<div class="alert alert-info">

**Note:**    WARNING: This notebook uses a new, early release Chameleon python API. The notebook and API will likely be updated without notice. To request a new python feature, please create a help desk ticket. 
    

</div>

## Tutorial: Stitching Networks between Chameleon Sites

This notebook demonstrates how to use ExoGENI to stitch networks between Chameleon sites. It will walk through reserving and configuring stitchable networks on each Chameleon site and using the xoStitch tool to create a layer2 circuit between the networks. 

#### Modules 

- [Reserve Mulitple Resources](../modules-python/reservations/reserve_multiple_resources.ipynb)
- [Get Lease by Name](../modules-python/reservations/get_lease_by_name.ipynb)
- [Get Reservation](../modules-python/reservations/get_reservations_from_lease.ipynb)
- [Create Server](../modules-python/servers/create_server.ipynb)
- [Delete Server](../modules-python/servers/delete_server.ipynb)
- [Create Network](../modules-python/network/create_network.ipynb)
- [Delete Network](../modules-python/network/delete_network.ipynb)
- [Create Subnet](../modules-python/network/create_subnet.ipynb)
- [Delete Subnet](../modules-python/network/delete_subnet.ipynb)
- [Get Network by Name](../modules-python/network/get_network_by_name.ipynb)






## Tutorial: 

#### Configure the Environment

Import the chi example API calls, set the project name and region, and set various names and attributes to use in the tutorial. 

In [1]:
import json
import os
import chi

from chi.server import *
from chi.lease import *
from chi.network import *

from datetime import datetime, timedelta
from dateutil import tz

BLAZAR_TIME_FORMAT = '%Y-%m-%d %H:%M'
VERBOSE=False  #Set to true for extra output

#Config with your project and site
chi.set('project_name', 'CH-822154') # Replace with your project name

#Insert keypair name
key_name = 'my_chameleon_key'  # Change to your keypair

#GENI Pem file
geni_pem_file='/home/pruth/work/geni-pruth.pem'

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")
server_name = username+'Server'
network_name = username+'Net'
subnet_name = username+'subnet'
router_name = username+'Router'
lease_name = username+'Lease'

#Server Config
image_name='CC-CentOS7'
flavor_name='baremetal'
node_type="compute_haswell"
server_count=1

#Network Config
physical_network='exogeni'
cidr='192.168.42.0/24'

## Create Chicago Network and Server

#### Create a Lease at Chicago

In [2]:
#Set the region
chi.use_site('CHI@UC')    # Optional, defaults to 'CHI@UC'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
#add_node_reservation(reservation_list, count=server_count, node_type=node_type)
add_network_reservation(reservation_list, network_name=network_name, physical_network=physical_network)
#add_fip_reservation(reservation_list, count=1)

# Create the lease
chi.blazar().lease.create(name=lease_name, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

Now using CHI@UC:
URL: https://chi.uc.chameleoncloud.org
Location: Argonne National Laboratory, Lemont, Illinois, USA
Support contact: help@chameleoncloud.org


{'status': 'PENDING',
 'user_id': '02276834a17c4ca205d49eaa8c984eeaefc24c153f8a62bc16e5f880e7cce607',
 'name': 'pruthLease',
 'end_date': '2021-02-18T18:11:00.000000',
 'reservations': [{'status': 'pending',
   'lease_id': '0480135d-6c6f-46e3-9ea3-8542cbb5c9be',
   'resource_id': '24409eca-3d34-4cee-9532-7815bd4b808a',
   'created_at': '2021-02-17 18:11:48',
   'updated_at': '2021-02-17 18:11:48',
   'missing_resources': False,
   'id': 'f5c75d98-3115-4889-b2bb-3e5acfd22f95',
   'resource_type': 'network',
   'resources_changed': False}],
 'created_at': '2021-02-17 18:11:48',
 'updated_at': '2021-02-17 18:11:48',
 'events': [{'status': 'UNDONE',
   'lease_id': '0480135d-6c6f-46e3-9ea3-8542cbb5c9be',
   'event_type': 'end_lease',
   'created_at': '2021-02-17 18:11:48',
   'updated_at': None,
   'time': '2021-02-18T18:11:00.000000',
   'id': '23dd4a22-6c5e-494f-af89-63c7df197a13'},
  {'status': 'UNDONE',
   'lease_id': '0480135d-6c6f-46e3-9ea3-8542cbb5c9be',
   'event_type': 'start_lease

In [3]:
#Get the lease by name
uc_lease = get_lease(get_lease_id(lease_name))
    
#Print the lease info
if VERBOSE:
    print(json.dumps(uc_lease, indent=2))
else:
    print('uc_lease: ' + uc_lease['name'] + ', ' + uc_lease['id'])

uc_lease: pruthLease, 0480135d-6c6f-46e3-9ea3-8542cbb5c9be


#### Get the Reservations

Each lease contains one or more reservations. The individual reservation IDs are required to instantiate resources. You can [get the lease](../modules-python/reservations/get_lease_by_name.ipynb) and separate the reservation IDs for compute, network, and floating IPs using the technique below.

In [6]:
#Get the lease by name
uc_lease = get_lease(get_lease_id(lease_name))

#uc_compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', uc_lease['reservations']))[0]['id']
uc_network_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'network', uc_lease['reservations']))[0]['id']
#uc_floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', uc_lease['reservations']))[0]['id']

#print("uc_compute_reservation_id: " + uc_compute_reservation_id)
print("uc_network_reservation_id: " + uc_network_reservation_id)
#print("uc_floatingip_reservation_id: " + uc_floatingip_reservation_id)

uc_network_reservation_id: f5c75d98-3115-4889-b2bb-3e5acfd22f95


#### Get the Network

Getting the network is not required for the remainder of the tutorial. However, it is a good test to see if your network reservation has become active. The [get_network_by_name](../modules-python/network/get_network_by_name.ipynb) call will fail if a network with that name does not yet exits. It will also fail if a network with the same name already exists (likely from a previous run of this notebook).

In [8]:
chi.use_site('CHI@UC')    # Optional, defaults to 'CHI@UC'

#Get the network
uc_network = get_network(get_network_id(network_name))
uc_network_id = uc_network['id']
uc_network_vlan = uc_network['provider:segmentation_id']

if VERBOSE:
    print(json.dumps(uc_network, indent=2))
else:
    print('uc_network: ' + uc_network['name'] + ', ' + uc_network['id'])
    print('uc segmenation ID/VLAN: ' + str(uc_network_vlan))

Now using CHI@UC:
URL: https://chi.uc.chameleoncloud.org
Location: Argonne National Laboratory, Lemont, Illinois, USA
Support contact: help@chameleoncloud.org
uc_network: pruthNet, 17dc0141-d3d4-4315-9a6c-2fe9808a138b
uc segmenation ID/VLAN: 3291


#### Add a subnet

[Adds a subnet](../modules-python/network/add_subnet.ipynb) to the reserved network. 

In [9]:
subnet = create_subnet(subnet_name, uc_network_id, cidr=cidr)
#print(json.dumps(subnet, indent=2))

In [10]:
if VERBOSE:
    print(json.dumps(subnet, indent=2))
else:
    print('Subnet: ' + subnet['name'] + ', ' + subnet['id'])

Subnet: pruthsubnet, ff346dfb-f9ce-4f23-979e-88ed1d85d463


#### Add a Router

TODO: add links here

In [11]:
router = create_router(router_name, gw_network_name='public')
#print(json.dumps(router, indent=2))

if VERBOSE:
    print(json.dumps(router, indent=2))
else:
    print('Router: ' + router['name'] + ', ' + router['id'])

Router: pruthRouter, d9622757-d1c8-4914-8167-a67e0b826b26


#### Attach the Router and Subnet

TODO: Add links here

In [12]:
add_subnet_to_router(get_router_id(router_name), get_subnet_id(subnet_name))

{'network_id': '17dc0141-d3d4-4315-9a6c-2fe9808a138b',
 'tenant_id': 'ae76673270164b048b59d3bd30676721',
 'subnet_id': 'ff346dfb-f9ce-4f23-979e-88ed1d85d463',
 'subnet_ids': ['ff346dfb-f9ce-4f23-979e-88ed1d85d463'],
 'port_id': '59bb6aa7-eb2a-4aae-81b7-ad40783dbf66',
 'id': 'd9622757-d1c8-4914-8167-a67e0b826b26'}

## Create TACC Network and Server

#### Create a Lease at Chicago

In [13]:
node_type="compute_skylake"

#Set the region
chi.use_site('CHI@TACC')    # Optional, defaults to 'CHI@UC'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
#add_node_reservation(reservation_list, count=server_count, node_type=node_type)
add_network_reservation(reservation_list, network_name=network_name, physical_network=physical_network)
#add_fip_reservation(reservation_list, count=1)

# Create the lease
chi.blazar().lease.create(name=lease_name, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

Now using CHI@TACC:
URL: https://chi.tacc.chameleoncloud.org
Location: Austin, Texas, USA
Support contact: help@chameleoncloud.org


{'status': 'PENDING',
 'user_id': 'c4733c8a0aa37ceb846208d989674028cddf7e404930d6fd63eba6546d8884a6',
 'name': 'pruthLease',
 'end_date': '2021-02-18T18:14:00.000000',
 'reservations': [{'status': 'pending',
   'lease_id': 'e97cb69a-5ab5-4d5f-88a8-e4606c64a161',
   'resource_id': '2939de2e-bc11-4f0a-8689-04490dab586c',
   'created_at': '2021-02-17 18:14:33',
   'updated_at': '2021-02-17 18:14:33',
   'missing_resources': False,
   'id': '61e9d832-b35c-40a0-b2de-8fcf05be0a81',
   'resource_type': 'network',
   'resources_changed': False}],
 'created_at': '2021-02-17 18:14:33',
 'updated_at': '2021-02-17 18:14:33',
 'events': [{'status': 'UNDONE',
   'lease_id': 'e97cb69a-5ab5-4d5f-88a8-e4606c64a161',
   'event_type': 'end_lease',
   'created_at': '2021-02-17 18:14:33',
   'updated_at': None,
   'time': '2021-02-18T18:14:00.000000',
   'id': '17a9af86-481a-443f-a81d-243bb485bd62'},
  {'status': 'UNDONE',
   'lease_id': 'e97cb69a-5ab5-4d5f-88a8-e4606c64a161',
   'event_type': 'start_lease

In [14]:
#Get the lease by name
tacc_lease = get_lease(get_lease_id(lease_name))
    
#Print the lease info
if VERBOSE:
    print(json.dumps(tacc_lease, indent=2))
else:
    print('tacc_lease: ' + tacc_lease['name'] + ', ' + tacc_lease['id'])

tacc_lease: pruthLease, e97cb69a-5ab5-4d5f-88a8-e4606c64a161


#### Get the Reservations

Each lease contains one or more reservations. The individual reservation IDs are required to instantiate resources. You can [get the lease](../modules-python/reservations/get_lease_by_name.ipynb) and separate the reservation IDs for compute, network, and floating IPs using the technique below.

In [15]:
#Get the lease by name
tacc_lease = get_lease(get_lease_id(lease_name))

#tacc_compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', tacc_lease['reservations']))[0]['id']
tacc_network_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'network', tacc_lease['reservations']))[0]['id']
#tacc_floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', tacc_lease['reservations']))[0]['id']

#print("tacc_compute_reservation_id: " + tacc_compute_reservation_id)
print("tacc_network_reservation_id: " + tacc_network_reservation_id)
#print("tacc_floatingip_reservation_id: " + tacc_floatingip_reservation_id)

tacc_network_reservation_id: 61e9d832-b35c-40a0-b2de-8fcf05be0a81


#### Get the Network

Getting the network is not required for the remainder of the tutorial. However, it is a good test to see if your network reservation has become active. The [get_network_by_name](../modules-python/network/get_network_by_name.ipynb) call will fail if a network with that name does not yet exits. It will also fail if a network with the same name already exists (likely from a previous run of this notebook).

In [16]:
chi.use_site('CHI@TACC')    # Optional, defaults to 'CHI@UC'

#Get the network
tacc_network = get_network(get_network_id(network_name))
tacc_network_id = tacc_network['id']
tacc_network_vlan = tacc_network['provider:segmentation_id']

if VERBOSE:
    print(json.dumps(tacc_network, indent=2))
else:
    print('tacc_network: ' + tacc_network['name'] + ', ' + tacc_network['id'])
    print('tacc segmenation ID/VLAN: ' + str(tacc_network_vlan))

Now using CHI@TACC:
URL: https://chi.tacc.chameleoncloud.org
Location: Austin, Texas, USA
Support contact: help@chameleoncloud.org
tacc_network: pruthNet, a01d6fb4-89a9-456d-a4bb-aa3a8c94c354
tacc segmenation ID/VLAN: 3503


#### Add a subnet

[Adds a subnet](../modules-python/network/add_subnet.ipynb) to the reserved network. 

In [17]:
subnet = create_subnet(subnet_name, tacc_network_id, cidr=cidr)
#print(json.dumps(subnet, indent=2))

In [18]:
if VERBOSE:
    print(json.dumps(subnet, indent=2))
else:
    print('Subnet: ' + subnet['name'] + ', ' + subnet['id'])

Subnet: pruthsubnet, 9f94c756-1d8a-42b7-abe7-24c8ff02cb31


#### Add a Router

TODO: add links here

In [19]:
router = create_router(router_name, gw_network_name='public')
#print(json.dumps(router, indent=2))

if VERBOSE:
    print(json.dumps(router, indent=2))
else:
    print('Router: ' + router['name'] + ', ' + router['id'])

Router: pruthRouter, a71fb8c6-16d1-4223-9e04-22336307397d


#### Attach the Router and Subnet

TODO: Add links here

In [20]:
add_subnet_to_router(get_router_id(router_name), get_subnet_id(subnet_name))

{'network_id': 'a01d6fb4-89a9-456d-a4bb-aa3a8c94c354',
 'tenant_id': 'a400724e818d40cbba1a5c6b5e714462',
 'subnet_id': '9f94c756-1d8a-42b7-abe7-24c8ff02cb31',
 'subnet_ids': ['9f94c756-1d8a-42b7-abe7-24c8ff02cb31'],
 'port_id': 'a9c21e1d-687b-4dca-bcd7-d16376b03201',
 'id': 'a71fb8c6-16d1-4223-9e04-22336307397d'}

## Start the Servers

#### Start the UC Server

Use the compute_reservation_id to [create the server](../modules-python/servers/create_server.ipynb).

In [ ]:
chi.use_site('CHI@UC')  
#create the server
server = create_server(server_name, 
                       reservation_id=uc_compute_reservation_id, 
                       key_name=key_name, 
                       network_name=network_name, 
                       image_name=image_name, 
                       flavor_name=flavor_name)


#### Associate the Floating IP   
TODO: need to find floating_ip from the reservation that was just made

In [ ]:
chi.use_site('CHI@UC')  
floating_ip = associate_floating_ip(server_name)

print('Floating IP: ' + str(floating_ip))

#### Start the TACC Server

Use the compute_reservation_id to [create the server](../modules-python/servers/create_server.ipynb).

In [ ]:
chi.use_site('CHI@TACC')

print("server_name " + server_name)
print("tacc_compute_reservation_id " + tacc_compute_reservation_id)
print("key_name " + key_name) 
print("network_name " + network_name)
print("image_name " + image_name)
print("flavor_name " + flavor_name)

#create the server
server = create_server(server_name, 
                       reservation_id=tacc_compute_reservation_id, 
                       key_name=key_name, 
                       network_name=network_name, 
                       image_name=image_name, 
                       flavor_name=flavor_name)


#### Associate the Floating IP   
TODO: need to find floating_ip from the reservation that was just made

In [ ]:
chi.use_site('CHI@TACC')  

floating_ip = associate_floating_ip(server_name)

print('Floating IP: ' + str(floating_ip))

## Stitch the Circuit using ExoGENI

Note: The ExoGENI
steps require a valid GENI certificate at the path specified and a public/private keypair in ~/.ssh (run ssh-keygen with default inputs)

#### Create the Circuit

In [21]:
%%script env uc_vlan="$uc_network_vlan" tacc_vlan="$tacc_network_vlan" geni_pem="$geni_pem_file" bash

echo 'uc_vlan ' $uc_vlan ', tacc_vlan ' $tacc_vlan ', geni_pem ' $geni_pem
xoStitch create -sp1 uc -vlan1 $uc_vlan -sp2 tacc -vlan2 $tacc_vlan -c $geni_pem

uc_vlan  3291 , tacc_vlan  3503 , geni_pem  /home/pruth/work/geni-pruth.pem
Opening certificate /home/pruth/work/geni-pruth.pem and key /home/pruth/work/geni-pruth.pem


log4j:WARN No appenders could be found for logger (org.renci.ahab.libndl.LIBNDL).
log4j:WARN Please initialize the log4j system properly.


#### Check the Status of the Circuit

In [ ]:
%%script env uc_vlan="$uc_network_vlan" tacc_vlan="$tacc_network_vlan" geni_pem="$geni_pem_file" bash

echo 'uc_vlan ' $uc_vlan ', tacc_vlan ' $tacc_vlan ', geni_pem ' $geni_pem
xoStitch status -sp1 uc -vlan1 $uc_vlan -sp2 tacc -vlan2 $tacc_vlan -c $geni_pem

## Clean Up Resources

### Delete Stitched Circuit using ExoGENI

In [ ]:
%%script env uc_vlan="$uc_network_vlan" tacc_vlan="$tacc_network_vlan" geni_pem="$geni_pem_file" bash

echo 'uc_vlan ' $uc_vlan ', tacc_vlan ' $tacc_vlan ', geni_pem ' $geni_pem
xoStitch delete -sp1 uc -vlan1 $uc_vlan -sp2 tacc -vlan2 $tacc_vlan -c $geni_pem

### Delete TACC Resources

[Delete the server](../modules-python/servers/delete_server.ipynb) using its name.

In [ ]:
chi.use_site('region_name', 'CHI@TACC')

In [ ]:
delete_server_by_name(server_name)

#### De-configure Network
TODO: break up into steps

In [ ]:
detach_router_by_name(router_name=router_name, subnet_name=subnet_name)

In [ ]:
delete_router_by_name(router_name)

In [ ]:
delete_subnet_by_name(subnet_name)

In [ ]:
delete_network_by_name(network_name)

#### Release Lease

In [ ]:
delete_lease_by_name(lease_name)

### Delete UC Resources

[Delete the server](../modules-python/servers/delete_server.ipynb) using its name.

In [ ]:
chi.use_site('region_name', 'CHI@UC')

In [ ]:
delete_server_by_name(server_name)

#### De-configure Network
TODO: break up into steps

In [ ]:
detach_router_by_name(router_name=router_name, subnet_name=subnet_name)

In [ ]:
delete_router_by_name(router_name)

In [ ]:
delete_subnet_by_name(subnet_name)

In [ ]:
delete_network_by_name(network_name)

#### Release Lease

In [ ]:
delete_lease_by_name(lease_name)